In [1]:
"""
SHA-256 PREIMAGE SOLVER - VERSION CORREGIDA (Big-Endian + SHR Fix)
- Implementacion completa de 64 rondas con padding correcto
- Correccion de endianness al armar las words del bloque (big-endian por word)
- Correccion CRITICA de SHR() en representacion LSB-first
"""

import subprocess
import sys
import hashlib
import time

try:
    from pysat.solvers import Glucose4
except ImportError:
    print("Instalando python-sat...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "python-sat"])
    from pysat.solvers import Glucose4

# Constantes SHA-256
H_INIT = [
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
]

K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]


class Quaternion:
    """Representa 32 bits como variables SAT (LSB-first)"""
    def __init__(self, variables):
        if len(variables) != 32:
            raise ValueError("Quaternion debe tener 32 bits")
        self.bits = variables

    def __getitem__(self, idx):
        return self.bits[idx]

    def __iter__(self):
        return iter(self.bits)

    def __len__(self):
        return 32

    def to_int(self, model):
        """
        Convierte un Quaternion (LSB-first) a entero.
        bits[0] = bit 0 (LSB), bits[31] = bit 31 (MSB)
        """
        result = 0
        for i, var in enumerate(self.bits):
            if var > 0 and model[var - 1] > 0:
                result |= (1 << i)
            elif var < 0 and model[abs(var) - 1] < 0:
                result |= (1 << i)
        return result


class SHA256_CNF_Builder:
    """Constructor del circuito CNF"""
    def __init__(self):
        self.clauses = []
        self.var_counter = 1

    def new_var(self):
        var = self.var_counter
        self.var_counter += 1
        return var

    def new_quaternion(self):
        return Quaternion([self.new_var() for _ in range(32)])

    def add_clause(self, clause):
        self.clauses.append(clause)

    def xor_gate(self, a, b, out):
        self.add_clause([-a, -b, -out])
        self.add_clause([a, b, -out])
        self.add_clause([-a, b, out])
        self.add_clause([a, -b, out])

    def and_gate(self, a, b, out):
        self.add_clause([-out, a])
        self.add_clause([-out, b])
        self.add_clause([-a, -b, out])

    def or_gate(self, a, b, out):
        self.add_clause([a, b, -out])
        self.add_clause([-a, out])
        self.add_clause([-b, out])

    def ch_gate(self, e, f, g, out):
        self.add_clause([-e, -f, out])
        self.add_clause([-e, f, -out])
        self.add_clause([e, -g, out])
        self.add_clause([e, g, -out])

    def maj_gate(self, a, b, c, out):
        self.add_clause([-a, -b, out])
        self.add_clause([-a, -c, out])
        self.add_clause([-b, -c, out])
        self.add_clause([a, b, -out])
        self.add_clause([a, c, -out])
        self.add_clause([b, c, -out])

    def full_adder(self, a, b, cin, sum_out, cout):
        temp_xor = self.new_var()
        self.xor_gate(a, b, temp_xor)
        self.xor_gate(temp_xor, cin, sum_out)

        temp_and1 = self.new_var()
        temp_and2 = self.new_var()
        self.and_gate(a, b, temp_and1)
        self.and_gate(temp_xor, cin, temp_and2)
        self.or_gate(temp_and1, temp_and2, cout)

    def add32(self, a_quat, b_quat, out_quat):
        """
        Suma de 32 bits en LSB-first:
        i=0 (LSB) -> i=31 (MSB)
        """
        carry = self.new_var()
        self.add_clause([-carry])  # carry inicial = 0

        for i in range(32):
            new_carry = self.new_var()
            self.full_adder(a_quat[i], b_quat[i], carry, out_quat[i], new_carry)
            carry = new_carry

    def add32_multi(self, quats, out_quat):
        if len(quats) == 0:
            raise ValueError("Necesito al menos un Quaternion")

        if len(quats) == 1:
            for i in range(32):
                self.add_clause([quats[0][i], -out_quat[i]])
                self.add_clause([-quats[0][i], out_quat[i]])
            return

        temp = self.new_quaternion()
        self.add32(quats[0], quats[1], temp)

        for i in range(2, len(quats)):
            new_temp = self.new_quaternion()
            self.add32(temp, quats[i], new_temp)
            temp = new_temp

        for i in range(32):
            self.add_clause([temp[i], -out_quat[i]])
            self.add_clause([-temp[i], out_quat[i]])

    def rotr(self, quat, n):
        """
        Rotación a la derecha en LSB-first:
        out[i] = in[(i+n) % 32]
        """
        return Quaternion([quat[(i + n) % 32] for i in range(32)])

    def shr(self, quat, n):
        """
        Shift a la derecha en LSB-first (CORREGIDO):
        out[i] = in[i+n], rellena con 0 cuando i+n >= 32
        """
        zero = self.new_var()
        self.add_clause([-zero])  # zero = 0

        shifted = []
        for i in range(32):
            src = i + n
            if src >= 32:
                shifted.append(zero)
            else:
                shifted.append(quat[src])
        return Quaternion(shifted)

    def xor3_quaternion(self, qa, qb, qc, out_quat):
        for i in range(32):
            temp = self.new_var()
            self.xor_gate(qa[i], qb[i], temp)
            self.xor_gate(temp, qc[i], out_quat[i])

    def sigma0(self, quat):
        r2 = self.rotr(quat, 2)
        r13 = self.rotr(quat, 13)
        r22 = self.rotr(quat, 22)
        result = self.new_quaternion()
        self.xor3_quaternion(r2, r13, r22, result)
        return result

    def sigma1(self, quat):
        r6 = self.rotr(quat, 6)
        r11 = self.rotr(quat, 11)
        r25 = self.rotr(quat, 25)
        result = self.new_quaternion()
        self.xor3_quaternion(r6, r11, r25, result)
        return result

    def sigma_lower0(self, quat):
        r7 = self.rotr(quat, 7)
        r18 = self.rotr(quat, 18)
        s3 = self.shr(quat, 3)
        result = self.new_quaternion()
        self.xor3_quaternion(r7, r18, s3, result)
        return result

    def sigma_lower1(self, quat):
        r17 = self.rotr(quat, 17)
        r19 = self.rotr(quat, 19)
        s10 = self.shr(quat, 10)
        result = self.new_quaternion()
        self.xor3_quaternion(r17, r19, s10, result)
        return result

    def quaternion_from_constant(self, value):
        """
        Constante de 32 bits a Quaternion LSB-first
        """
        quat = self.new_quaternion()
        for i in range(32):
            bit_val = (value >> i) & 1
            self.add_clause([quat[i]] if bit_val else [-quat[i]])
        return quat

    def build_sha256_circuit(self, message_block, target_hash_prefix_bits=0):
        print("\n[FASE 1] Construyendo circuito SHA-256 (64 rondas)...")

        # Expansion W[0..63]
        print("  -> Expansion del mensaje...")
        W = list(message_block)

        for t in range(16, 64):
            s1 = self.sigma_lower1(W[t - 2])
            s0 = self.sigma_lower0(W[t - 15])
            wt = self.new_quaternion()
            self.add32_multi([s1, W[t - 7], s0, W[t - 16]], wt)
            W.append(wt)

        print(f"  -> {len(W)} palabras expandidas")

        # Inicializar estado
        print("  -> Inicializando estado...")
        a = self.quaternion_from_constant(H_INIT[0])
        b = self.quaternion_from_constant(H_INIT[1])
        c = self.quaternion_from_constant(H_INIT[2])
        d = self.quaternion_from_constant(H_INIT[3])
        e = self.quaternion_from_constant(H_INIT[4])
        f = self.quaternion_from_constant(H_INIT[5])
        g = self.quaternion_from_constant(H_INIT[6])
        h = self.quaternion_from_constant(H_INIT[7])

        # 64 rondas
        print("  -> Ejecutando 64 rondas...")
        for t in range(64):
            if t % 16 == 0 and t > 0:
                print(f"     Ronda {t}/64")

            # T1 = h + Sigma1(e) + Ch(e,f,g) + K[t] + W[t]
            sigma1_e = self.sigma1(e)

            ch_result = self.new_quaternion()
            for i in range(32):
                self.ch_gate(e[i], f[i], g[i], ch_result[i])

            K_t = self.quaternion_from_constant(K[t])

            T1 = self.new_quaternion()
            self.add32_multi([h, sigma1_e, ch_result, K_t, W[t]], T1)

            # T2 = Sigma0(a) + Maj(a,b,c)
            sigma0_a = self.sigma0(a)

            maj_result = self.new_quaternion()
            for i in range(32):
                self.maj_gate(a[i], b[i], c[i], maj_result[i])

            T2 = self.new_quaternion()
            self.add32_multi([sigma0_a, maj_result], T2)

            # Actualizar
            h = g
            g = f
            f = e
            e = self.new_quaternion()
            self.add32_multi([d, T1], e)
            d = c
            c = b
            b = a
            a = self.new_quaternion()
            self.add32_multi([T1, T2], a)

        print("  -> 64 rondas completadas")

        # Hash final
        print("  -> Calculando hash final...")
        H_final = []
        working_vars = [a, b, c, d, e, f, g, h]

        for i in range(8):
            H_i = self.new_quaternion()
            H_init_quat = self.quaternion_from_constant(H_INIT[i])
            self.add32_multi([H_init_quat, working_vars[i]], H_i)
            H_final.append(H_i)

        print("  -> Hash calculado")

        # Constraints del prefijo (big-endian a nivel de bits del digest)
        if target_hash_prefix_bits > 0:
            print(f"\n[FASE 2] Aplicando constraint de {target_hash_prefix_bits} bits...")
            bit_count = 0
            for word_idx in range(8):
                if bit_count >= target_hash_prefix_bits:
                    break
                # En LSB-first, MSB es bit 31, así que imponemos de 31 -> 0
                for bit_idx in range(31, -1, -1):
                    if bit_count >= target_hash_prefix_bits:
                        break
                    self.add_clause([-H_final[word_idx][bit_idx]])  # forzar a 0
                    bit_count += 1
            print(f"  -> {target_hash_prefix_bits} bits forzados a 0")

        return H_final


def find_preimage():
    print("\n" + "=" * 70)
    print("SHA-256 PREIMAGE SOLVER - Busqueda de preimagen")
    print("=" * 70)

    # Configuracion
    MESSAGE_LENGTH_BYTES = 4
    TARGET_PREFIX_BITS = 8  # Hash debe empezar con 0x00

    # Crear padding SHA-256 para un solo bloque (64 bytes)
    padding_bytes = bytearray(64)
    padding_bytes[MESSAGE_LENGTH_BYTES] = 0x80
    length_bits = MESSAGE_LENGTH_BYTES * 8
    padding_bytes[56:64] = length_bits.to_bytes(8, byteorder='big')

    print(f"\n[CONFIG]")
    print(f"  Mensaje: {MESSAGE_LENGTH_BYTES} bytes libres")
    print(f"  Objetivo: Hash que empiece con {TARGET_PREFIX_BITS} bits en 0")

    # Construir circuito
    builder = SHA256_CNF_Builder()

    print(f"\n[CONSTRUCCION] Creando circuito...")

    message_block = []
    free_vars = []

    # Armado correcto del bloque:
    # - W0 usa bytes [0..3] en big-endian dentro de la word
    # - pero el Quaternion almacena bits LSB-first
    for word_idx in range(16):
        quat = builder.new_quaternion()

        for byte_idx in range(4):
            global_byte_idx = word_idx * 4 + byte_idx

            # SHA-256: word big-endian => byte 0 es el MSB del word (bits 24..31)
            # En LSB-first, esos bits son indices 24..31, base = 24
            dest_base = (3 - byte_idx) * 8

            if global_byte_idx < MESSAGE_LENGTH_BYTES:
                # Byte libre: bit0 (LSB del byte) -> dest_base+0
                for bit_idx in range(8):
                    free_vars.append(quat[dest_base + bit_idx])
            else:
                # Padding fijo
                byte_val = padding_bytes[global_byte_idx]
                for bit_idx in range(8):
                    bit_val = (byte_val >> bit_idx) & 1
                    var = quat[dest_base + bit_idx]
                    builder.add_clause([var] if bit_val else [-var])

        message_block.append(quat)

    print(f"  Variables libres: {len(free_vars)}")
    print(f"  Bits fijos (padding): {512 - len(free_vars)}")

    H_final = builder.build_sha256_circuit(
        message_block,
        target_hash_prefix_bits=TARGET_PREFIX_BITS
    )

    print(f"\n[STATS]")
    print(f"  Variables SAT: {builder.var_counter - 1:,}")
    print(f"  Clausulas CNF: {len(builder.clauses):,}")

    # Resolver
    print(f"\n" + "=" * 70)
    print("[SAT SOLVER] Resolviendo...")
    print("=" * 70)

    solver = Glucose4()

    print("\n  Añadiendo clausulas...")
    for clause in builder.clauses:
        solver.add_clause(clause)

    print("  Ejecutando solver (puede tardar minutos)...\n")

    start_time = time.time()
    is_sat = solver.solve()
    elapsed = time.time() - start_time

    print(f"\n  Tiempo: {elapsed:.2f}s")

    if not is_sat:
        print("\n" + "=" * 70)
        print("NO SE ENCONTRO SOLUCION (UNSAT)")
        print("=" * 70)
        print("\nPosibles causas:")
        print("  1. Prefijo demasiado restrictivo")
        print("  2. Error en el circuito CNF")
        return

    print("\n" + "=" * 70)
    print("SOLUCION ENCONTRADA!")
    print("=" * 70)

    model = solver.get_model()

    # Reconstruir bloque completo (64 bytes) desde el modelo SAT
    message_bytes = bytearray()
    for word_idx in range(16):
        word_val = message_block[word_idx].to_int(model)
        message_bytes += word_val.to_bytes(4, byteorder='big')

    original_message = bytes(message_bytes[:MESSAGE_LENGTH_BYTES])

    print(f"\nMensaje encontrado ({MESSAGE_LENGTH_BYTES} bytes):")
    print(f"  Hex: {original_message.hex()}")
    print(f"  Dec: {int.from_bytes(original_message, 'big')}")

    # Verificar con hashlib
    real_hash = hashlib.sha256(original_message).hexdigest()

    print(f"\nHash SHA-256 (hashlib):")
    print(f"  {real_hash}")

    expected_prefix = '0' * (TARGET_PREFIX_BITS // 4)
    matches = real_hash.startswith(expected_prefix)

    print(f"\nBloque completo (con padding):")
    print(f"  {message_bytes.hex()}")

    # Hash completo desde SAT
    print(f"\n[DEBUG] Verificando hash desde variables SAT:")
    hash_from_sat = ""
    for word_idx in range(8):
        word_val = H_final[word_idx].to_int(model)
        hash_from_sat += f"{word_val:08x}"

    print(f"  Hash desde SAT   : {hash_from_sat}")
    print(f"  Hash desde hashlib: {real_hash}")
    print(f"  Coinciden        : {hash_from_sat == real_hash}")

    if matches and (hash_from_sat == real_hash):
        print(f"\n" + "=" * 70)
        print("VERIFICACION EXITOSA!")
        print("=" * 70)
        print(f"El hash comienza con '{expected_prefix}' ({TARGET_PREFIX_BITS} bits en 0)")
        print(f"\nPREIMAGEN ENCONTRADA CORRECTAMENTE!")
    else:
        print(f"\n" + "=" * 70)
        print("ADVERTENCIA: AUN HAY DISCREPANCIA")
        print("=" * 70)
        if not matches:
            print(f"- hashlib NO cumple prefijo: esperado {expected_prefix}..., obtenido {real_hash[:len(expected_prefix)]}...")
        if hash_from_sat != real_hash:
            print("- El hash del circuito SAT no coincide con hashlib.")
            print("  Esto indica que aún queda algún detalle de modelado/endianness por corregir.")


if __name__ == "__main__":
    find_preimage()
    print("\n" + "=" * 70)
    print("Ejecucion completada")
    print("=" * 70 + "\n")

Instalando python-sat...

SHA-256 PREIMAGE SOLVER - Busqueda de preimagen

[CONFIG]
  Mensaje: 4 bytes libres
  Objetivo: Hash que empiece con 8 bits en 0

[CONSTRUCCION] Creando circuito...
  Variables libres: 32
  Bits fijos (padding): 480

[FASE 1] Construyendo circuito SHA-256 (64 rondas)...
  -> Expansion del mensaje...
  -> 64 palabras expandidas
  -> Inicializando estado...
  -> Ejecutando 64 rondas...
     Ronda 16/64
     Ronda 32/64
     Ronda 48/64
  -> 64 rondas completadas
  -> Calculando hash final...
  -> Hash calculado

[FASE 2] Aplicando constraint de 8 bits...
  -> 8 bits forzados a 0

[STATS]
  Variables SAT: 128,184
  Clausulas CNF: 427,936

[SAT SOLVER] Resolviendo...

  Añadiendo clausulas...
  Ejecutando solver (puede tardar minutos)...


  Tiempo: 19.88s

SOLUCION ENCONTRADA!

Mensaje encontrado (4 bytes):
  Hex: 5bbfb89d
  Dec: 1539291293

Hash SHA-256 (hashlib):
  00ec013775edb05db44b992e7a854a20393f5d2a73a1978547824e9a19753ff6

Bloque completo (con padding):
